# Generating Mixtures for Mosaic Variant Power Analysis

## Summary
Generating mixtures and subsets of Illumina 2X150 bp reads from HG002 and HG003 for a somatic variant caller power analysis. 

HG002 300X Ill bam has 6,263,741,684 reads
HG003 300X Ill bam has 6,015,660,400

Number of reads per bam file calculated using `samtools idxstats /Volumes/giab/data/alignment/AshkenazimTrio/Illumina/NHGRI_Illumina300X_AJtrio_novoalign_bams/HG002.hs37d5.300x.bam | awk '{s+=$3+$4} END {print s}'`

The HG002 reads represent the somatic or mosaic allele fraction.  
The HG003 reads represent the normal allele fraction.

Table: Number of reads (in millons) from Ill 300X dataset used in each mixture. 

| AF   | Tumor |       | Normal |
|------|-------|-------|--------|
|      | HG002 | HG003 | HG003  |
| 0.5  | 3000  |    0  | 3000   |
| 0.4  | 2400  |  600  | 3000   |
| 0.3  | 1800  | 1200  | 3000   |
| 0.2  | 1200  | 1800  | 3000   |
| 0.1  |  600  | 2400  | 3000   |
| 0.05 |  300  | 2700  | 3000   |
| 0.01 |   60  | 2940  | 3000   |
| 0    |    0  | 3000  | 3000   | 

## Approach  
The Ill 300X HG002 and HG003 datasets will be subset into files with different numbers of reads for using picard's `SplitSamByNumberOfReads` function, http://broadinstitute.github.io/picard/. 

1. The 300X Ill HG002 and HG003 Ill dataset will first be split into files with 150M reads.
1. One of these 150M read files for HG002 and HG003 will be further split into files with 30M read. 
1. The HG002 300X Ill dataset and one of the HG003 120M read files will be split into files with 6M reads. 
1. One of each of the HG002 and HG003 6M read files will be further split into 5 files with 1.2M reads.
1. The read subsets will be mixed to generate tumor bams with the allele fraction defined above.  




# Spliting 300X bams

## HG003 Split 1

In [16]:
%%bash
INBAM=/Volumes/giab/data/alignment/AshkenazimTrio/Illumina/NHGRI_Illumina300X_AJtrio_novoalign_bams/HG003.hs37d5.300x.bam
OUTDIR=/Volumes/giab/analysis/giab-mosaic-variants/data/HG003_split1
mkdir -p ${OUTDIR}

picard SplitSamByNumberOfReads \
    I=${INBAM} \
    TOTAL_READS=6015660400 \
    N_READS=150000000 \
    OUTPUT=${OUTDIR} \
    OUT_PREFIX=HG003-split1

INFO	2019-12-20 12:49:48	SplitSamByNumberOfReads	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    SplitSamByNumberOfReads -I /Volumes/giab/data/alignment/AshkenazimTrio/Illumina/NHGRI_Illumina300X_AJtrio_novoalign_bams/HG003.hs37d5.300x.bam -TOTAL_READS 6015660400 -N_READS 150000000 -OUTPUT /Volumes/giab/analysis/giab-mosaic-variants/data/HG003_split1 -OUT_PREFIX HG003-split1
**********


12:49:48.642 INFO  NativeLibraryLoader - Loading libgkl_compression.dylib from jar:file:/Users/nolson/anaconda3/share/picard-2.21.6-0/picard.jar!/com/intel/gkl/native/libgkl_compression.dylib
[Fri Dec 20 12:49:48 EST 2019] SplitSamByNumberOfReads INPUT=/Volumes/giab/data/alignment/AshkenazimTrio/Illumina/NHGRI_Illumina300X_A

## HG003 Split 2

In [18]:
%%bash
INBAM=/Volumes/giab/analysis/giab-mosaic-variants/data/HG003_split1/HG003-split1_0020.bam
OUTDIR=/Volumes/giab/analysis/giab-mosaic-variants/data/HG003_split2
mkdir -p ${OUTDIR}

picard SplitSamByNumberOfReads \
    I=${INBAM} \
    TOTAL_READS=150000000 \
    N_READS=30000000 \
    OUTPUT=${OUTDIR} \
    OUT_PREFIX=HG003-split2

INFO	2019-12-23 08:25:38	SplitSamByNumberOfReads	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    SplitSamByNumberOfReads -I /Volumes/giab/analysis/giab-mosaic-variants/data/HG003_split1/HG003-split1_0020.bam -TOTAL_READS 150000000 -N_READS 30000000 -OUTPUT /Volumes/giab/analysis/giab-mosaic-variants/data/HG003_split2 -OUT_PREFIX HG003-split2
**********


08:25:39.356 INFO  NativeLibraryLoader - Loading libgkl_compression.dylib from jar:file:/Users/nolson/anaconda3/share/picard-2.21.6-0/picard.jar!/com/intel/gkl/native/libgkl_compression.dylib
[Mon Dec 23 08:25:39 EST 2019] SplitSamByNumberOfReads INPUT=/Volumes/giab/analysis/giab-mosaic-variants/data/HG003_split1/HG003-split1_0020.bam SPLIT_TO_N_READS=300000

## HG002 Split 1

In [20]:
%%bash
INBAM=/Volumes/giab/data/alignment/AshkenazimTrio/Illumina/NHGRI_Illumina300X_AJtrio_novoalign_bams/HG002.hs37d5.300x.bam
OUTDIR=/Volumes/giab/analysis/giab-mosaic-variants/data/HG002_split1
mkdir -p ${OUTDIR}

ulimit -c unlimited
## Running detached to free up notebook for analysis
nohup picard SplitSamByNumberOfReads \
    I=${INBAM} \
    TOTAL_READS=6263741684 \
    N_READS=150000000 \
    OUTPUT=${OUTDIR} \
    OUT_PREFIX=HG002-split1 &> ${OUTDIR}/split.log &

## HG002 Split 2

In [ ]:
%%bash
INBAM=/Volumes/giab/analysis/giab-mosaic-variants/data/HG002_split1/HG002-split1_0020.bam
OUTDIR=/Volumes/giab/analysis/giab-mosaic-variants/data/HG002_split2
mkdir -p ${OUTDIR}

picard SplitSamByNumberOfReads \
    I=${INBAM} \
    TOTAL_READS=150000000 \
    N_READS=30000000 \
    OUTPUT=${OUTDIR} \
    OUT_PREFIX=HG002-split2

## Read Split Sanity Check

In [35]:
%%bash
WKDIR=/Volumes/giab/analysis/giab-mosaic-variants/data/
RCFILE=${WKDIR}/split_read_counts.csv

echo "file,read_count" > ${RCFILE}

for bam in ${WKDIR}/HG003_split{1,2}/*[[:digit:]].bam; do
    STATSFILE=${bam%.*}_idxstats.txt
    FNAME=$(basename ${bam}) 
    
    if [ ! -f "${bam}.bai" ]; then
        samtools index ${bam}
    fi

    if [ ! -f "${STATSFILE}" ]; then
        samtools idxstats ${bam} > ${STATSFILE}
    fi

    NREADS=$(awk '{s+=$3+$4} END {print s}' ${STATSFILE})
    echo $(basename ${bam}),${NREADS} >> ${RCFILE}
done


In [36]:
%%bash
WKDIR=/Volumes/giab/analysis/giab-mosaic-variants/data/
RCFILE=${WKDIR}/split_read_counts_hg002.csv

echo "file,read_count" > ${RCFILE}

for bam in ${WKDIR}/HG002_split{1,2}/*[[:digit:]].bam; do
    STATSFILE=${bam%.*}_idxstats.txt
    FNAME=$(basename ${bam}) 
    
    if [ ! -f "${bam}.bai" ]; then
        samtools index ${bam}
    fi

    if [ ! -f "${STATSFILE}" ]; then
        samtools idxstats ${bam} > ${STATSFILE}
    fi

    NREADS=$(awk '{s+=$3+$4} END {print s}' ${STATSFILE})
    echo $(basename ${bam}),${NREADS} >> ${RCFILE}
done

[E::hts_open_format] Failed to open file /Volumes/giab/analysis/giab-mosaic-variants/data//HG002_split2/*[[:digit:]].bam
samtools index: failed to open "/Volumes/giab/analysis/giab-mosaic-variants/data//HG002_split2/*[[:digit:]].bam": No such file or directory
bash: line 15: /Volumes/giab/analysis/giab-mosaic-variants/data//HG002_split2/*[[:digit:]]_idxstats.txt: No such file or directory
awk: can't open file /Volumes/giab/analysis/giab-mosaic-variants/data//HG002_split2/*[[:digit:]]_idxstats.txt
 source line number 1


# Make Turmor Mixtures

1. Change split file read group to prevent mixture issues
1. Combine appropriate subsets to make desired allele fractions
1. Sort and index combined bams
1. Upload files to DNAnexus 

## Fixing read groups
Bams currently do not have read groups.  
Using sample to indicate Turmor - HG002-HG003 mix.  
Using ID to indicate specific bam subset.  

In [33]:
%%bash
WKDIR=/Volumes/giab/analysis/giab-mosaic-variants/data/

for INBAM in ${WKDIR}/HG003_split{1,2}/HG003-split{1,2}_00{01..20}.bam; do
    
    if [ -f ${INBAM} ]; then
            SPLITID=$( basename ${INBAM%.bam})
            OUTBAM=${INBAM%.bam}_wrg.bam
            
            if [ ! -f "${OUTBAM}" ]; then
                picard AddOrReplaceReadGroups \
                    I=${INBAM} \
                    O=${OUTBAM} \
                    RGID=${SPLITID} \
                    RGLB=all \
                    RGPL=ILLUMINA \
                    RGPU=all \
                    RGSM=THG002-NHG003

                samtools index ${OUTBAM}
            fi
    fi

done

INFO	2019-12-23 09:22:06	AddOrReplaceReadGroups	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    AddOrReplaceReadGroups -I /Volumes/giab/analysis/giab-mosaic-variants/data//HG003_split2/HG003-split2_0001.bam -O /Volumes/giab/analysis/giab-mosaic-variants/data//HG003_split2/HG003-split2_0001_wrg.bam -RGID HG003-split2_0001 -RGLB all -RGPL ILLUMINA -RGPU all -RGSM THG002-NHG003
**********


09:22:06.586 INFO  NativeLibraryLoader - Loading libgkl_compression.dylib from jar:file:/Users/nolson/anaconda3/share/picard-2.21.6-0/picard.jar!/com/intel/gkl/native/libgkl_compression.dylib
[Mon Dec 23 09:22:06 EST 2019] AddOrReplaceReadGroups INPUT=/Volumes/giab/analysis/giab-mosaic-variants/data//HG003_split2/HG003-split

In [ ]:
%%bash
WKDIR=/Volumes/giab/analysis/giab-mosaic-variants/data/

for INBAM in ${WKDIR}/HG003_split1/HG003-split1_00{21..40}.bam; do
    
    if [ -f ${INBAM} ]; then
            SPLITID=$( basename ${INBAM%.bam})
            OUTBAM=${INBAM%.bam}_wrg.bam
            
            if [ ! -f "${OUTBAM}" ]; then
                picard AddOrReplaceReadGroups \
                    I=${INBAM} \
                    O=${OUTBAM} \
                    RGID=${SPLITID} \
                    RGLB=all \
                    RGPL=ILLUMINA \
                    RGPU=all \
                    RGSM=NHG003

                samtools index ${OUTBAM}
            fi
    fi

done

## Combining BAM files

### Normal Sample

In [ ]:
%%bash
WKDIR=Volumes/giab/analysis/giab-mosaic-variants/data/

echo ${WKDIR}/HG003_split1/HG003-split1_00{21..40}.bam > bam.lst
bamtools merge ${WKDIR}/mixtures/NHG003.bam 
samtools sort ${merged_bam} > sorted.bam
samtools index sorted.bam


In [ ]:
### Turmor Samples